In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.stats import linregress

# Function to calculate RSI
def calculate_rsi(data, window=14):
    delta = data['Adj Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Fetch S&P 500 monthly data for the last 20 years
sp500 = yf.download('^GSPC', period="20y", interval="1mo")

# Calculate RSI
sp500['RSI'] = calculate_rsi(sp500, window=14)

# Calculate the slope of the RSI for the last 5 months in each row
N = 5  # Number of months to calculate the slope
slopes = []  # List to store slope values

for i in range(len(sp500)):
    if i < N - 1:  # Not enough data to calculate the slope
        slopes.append(np.nan)
        continue
    
    # Linear regression over the last N RSI values
    y = sp500['RSI'].iloc[i-N+1:i+1].values
    x = np.arange(N)
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    slopes.append(slope)

# Add slopes to the DataFrame
sp500['RSI_Slope'] = slopes

# Drop rows with NaN values (the first few rows without enough data for RSI or slope calculation)
sp500.dropna(inplace=True)

# Save to CSV
sp500.to_csv('sp500_monthly_rsi_slope.csv')

print("Monthly RSI slope for the S&P 500 over the last 20 years saved to sp500_monthly_rsi_slope.csv.")


[*********************100%%**********************]  1 of 1 completed

Monthly RSI slope for the S&P 500 over the last 20 years saved to sp500_monthly_rsi_slope.csv.
